In [16]:
# import numpy as np
# import cv2
# import imutils

In [ ]:
# # load the image and convert it to grayscale
# def get_barcode(image_path):
#     # image = cv2.imread("D:/DTU-D/4/Fagprojekt/Data/photos/00a53a17-50db-4a3d-be1a-31ffd9537030/image.jpg")
#     # image = cv2.imread(r"D:\DTU-D\4\Fagprojekt\Data\photos\00dc11b6-bb75-4b9b-95c1-b4f247796b25\20201120_0832291.jpg")
#     image = cv2.imread(r"D:\DTU-D\4\Fagprojekt\Data\photos\00e98217-20c2-429a-aa1a-b8c183a84d24\5AB1D358-F313-4107-9272-AA7D08D52804.jpeg")
#     # image = cv2.imread(r"D:\DTU-D\4\Fagprojekt\Data\photos\0a000d87-9173-4bc3-a433-b77223437aa5\IMG_0329.JPG")
#     gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
#     # compute the Scharr gradient magnitude representation of the images
#     # in both the x and y direction
#     gradX = cv2.Sobel(gray, ddepth = cv2.CV_32F, dx = 1, dy = 0, ksize = -1)
#     gradY = cv2.Sobel(gray, ddepth = cv2.CV_32F, dx = 0, dy = 1, ksize = -1)
#     # subtract the y-gradient from the x-gradient
#     gradient = cv2.subtract(gradX, gradY)
#     gradient = cv2.convertScaleAbs(gradient)
#
#     # blur and threshold the image
#     blurred = cv2.blur(gradient, (9, 9))
#     (_, thresh) = cv2.threshold(blurred, 225, 255, cv2.THRESH_BINARY)
#
#     # construct a closing kernel and apply it to the thresholded image
#     kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (21, 7))
#     closed = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
#
#     # perform a series of erosions and dilations
#     closed = cv2.erode(closed, None, iterations = 4)
#     closed = cv2.dilate(closed, None, iterations = 4)
#
#     # find the contours in the thresholded image, then sort the contours
#     # by their area, keeping only the largest one
#     cnts = cv2.findContours(closed.copy(), cv2.RETR_EXTERNAL,
#                             cv2.CHAIN_APPROX_SIMPLE)
#     cnts = imutils.grab_contours(cnts)
#     c = sorted(cnts, key = cv2.contourArea, reverse = True)[0]
#
#     # compute the rotated bounding box of the largest contour
#     rect = cv2.minAreaRect(c)
#     box = cv2.cv.BoxPoints(rect) if imutils.is_cv2() else cv2.boxPoints(rect)
#     box = np.int0(box)
#
#     # draw a bounding box arounded the detected barcode and display the
#     # image
#     cv2.drawContours(image, [box], -1, (0, 255, 0), 3)
#     # cv2.imshow("Image", image)
#     # cv2.waitKey(0)
#     # cv2.imwrite("Image.jpg", image)
# get_barcode("")

In [4]:
# # Importing library
# import cv2
# from pyzbar.pyzbar import decode
#
# # Make one method to decode the barcode
# def BarcodeReader(image):
#
#     # read the image in numpy array using cv2
#     img = cv2.imread(image)
#
#     # Decode the barcode image
#     detectedBarcodes = decode(img)
#
#     # If not detected then print the message
#     if not detectedBarcodes:
#         print("Barcode Not Detected or your barcode is blank/corrupted!")
#     else:
#
#         # Traverse through all the detected barcodes in image
#         for barcode in detectedBarcodes:
#
#             # Locate the barcode position in image
#             (x, y, w, h) = barcode.rect
#
#             # Put the rectangle in image using
#             # cv2 to heighlight the barcode
#             cv2.rectangle(img, (x-10, y-10),
#                           (x + w+10, y + h+10),
#                           (255, 0, 0), 2)
#
#             if barcode.data!="":
#
#                 # Print the barcode data
#                 print(barcode.data)
#                 print(barcode.type)
#
#     #Display the image
#     # cv2.imshow("Image", img)
#     # cv2.waitKey(0)
#     # cv2.destroyAllWindows()
# BarcodeReader("D:/DTU-D/4/Fagprojekt/Data/photos/00a53a17-50db-4a3d-be1a-31ffd9537030/test.jpg")

KeyboardInterrupt: 

In [1]:
import cv2
from pyzbar.pyzbar import decode
from pyzbar.pyzbar import ZBarSymbol

pic_path = "D:/DTU-D/4/Fagprojekt/Data/photos/00a53a17-50db-4a3d-be1a-31ffd9537030/test.jpg"
# preprocessing using opencv
im = cv2.imread(pic_path, cv2.IMREAD_GRAYSCALE)
ret, bw_im = cv2.threshold(im, 127, 255, cv2.THRESH_BINARY)
# zbar
barcodes = decode(bw_im, symbols=[ZBarSymbol.CODE93])

In [1]:
import pandas as pd
import os
import numpy as np
from tqdm.notebook import tqdm
import cv2
from pyzbar.pyzbar import decode

rootPath = "D:/DTU-D/4/Fagprojekt/Data"
csvName = "labels.csv"
csvName2 = "labels2.csv"

data = pd.concat([
    pd.read_csv(os.path.join(rootPath,csvName)),
    pd.read_csv(os.path.join(rootPath,csvName2))
], ignore_index=True)
data.drop(["Unnamed: 0"],axis=1,inplace=True)

batch = 1
# data.drop(data.index[[1711]], inplace=True) #GIF
data.drop(data.index[[3033,3034,3035]], inplace=True) #Corrupted pictures
rows = data.shape[0]

In [2]:
# Preprocess image paths
new_paths = []
to_remove = []
for i in tqdm(range(rows)):
    image_path = data.iloc[i]['image_path'].lower()
    if image_path.endswith("gif") or image_path.endswith("pdf"):
        to_remove.append(i)
    # The replaced letters are NOT the same. There is a mismatch between the csv file and the actual picture name
    if "ä" in image_path:
        image_path = image_path.replace("ä", "ä")
    if "å" in image_path:
        image_path = image_path.replace("å", "å")
    if "ö" in image_path:
        image_path = image_path.replace("ö", "ö")
    path = os.path.join(rootPath,image_path)
    new_paths.append(path)
data['complete_path'] = new_paths
data.drop(data.index[to_remove], inplace=True) #gif or pdf
rows = data.shape[0]

  0%|          | 0/3498 [00:00<?, ?it/s]

In [5]:
results = {}
for i in tqdm(range(rows)):
    image_path = data.iloc[i]['complete_path']
    decoded_img = cv2.imdecode(np.fromfile(image_path, np.uint8), cv2.IMREAD_UNCHANGED)

    # Decode the barcode image
    detectedBarcodes = decode(decoded_img)

    if detectedBarcodes:
        results[i] = detectedBarcodes

  0%|          | 0/3211 [00:00<?, ?it/s]

In [23]:
# np.save("Results_04_06_2022_pyzbar", results)

# How to load:
# import numpy as np
# npzfile = np.load(r"D:\DTU-D\4\Fagprojekt\Hardware-identification-and-QA\Data exploration code\Results_04_06_2022_pyzbar.npy",
#                   allow_pickle=True)
# npzfile.item().get(12)

In [18]:
correct_read_ids = []
wat = 0
for key in results:
    result = results[key]
    ground_truth = data.iloc[key]['meter_number']

    for i in range(len(result)):
        reading = result[i].data.decode("utf-8")
        if ground_truth == reading:
            correct_read_ids.append(key)
            break

n_correct_readings = len(correct_read_ids)

In [35]:
# Not beautiful but it works
incorrect_readings = [key for key in results if key not in correct_read_ids]

In [61]:
idasdasd = 2136
print(data.iloc[idasdasd])
print(results[idasdasd])

image_path       photos/85f71492-dd12-44a0-86af-d8ec015ac14e/im...
meter_number                                                   NaN
project_id                    206fd8b6-db35-4382-a5f2-7df93e73f326
complete_path    D:/DTU-D/4/Fagprojekt/Data\photos/85f71492-dd1...
Name: 2137, dtype: object
[Decoded(data=b'5706567288837602', type='CODE128', rect=Rect(left=2598, top=1576, width=13, height=336), polygon=[Point(x=2598, y=1912), Point(x=2608, y=1912), Point(x=2611, y=1576), Point(x=2599, y=1576)], quality=6, orientation='LEFT')]
